In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import time
import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, confusion_matrix
import lightgbm as lgb
from math import radians, cos, sin, asin, sqrt
from tqdm import tqdm, tqdm_notebook
import pickle
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import math
from itertools import combinations
from sklearn.preprocessing import LabelEncoder
from gensim.models import Word2Vec
import json
import gc
import re
warnings.filterwarnings('ignore')
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
train = pd.read_csv('../data/first_round_training_data.csv')
test = pd.read_csv('../data/first_round_testing_data.csv')

In [4]:
len(train),len(test)

(6000, 6000)

In [5]:
features = [c for c in test.columns if c!='Group']
cat_feats = ['Attribute4',
'Attribute5',
'Attribute6',
'Attribute7',
'Attribute8',
'Attribute9',
'Attribute10',
'Parameter5',
'Parameter6',
'Parameter7',
'Parameter8',
'Parameter9',
'Parameter10']

use_cate = [c for c in cat_feats if 'Para' in c]  #使用到的6个变量
col_only_train = [c for c in test.columns if c != 'Group']  #训练集中的所有变量
all_feat = [c for c in train.columns if c != 'Quality_label']  #数据的所有特征
data = pd.concat([train,test])

def nnq_encode(data,en_col,use_col):
    data[en_col + '_nnq_of_' + use_col] = data[en_col].map( data.groupby([en_col])[use_col].nunique() )
    features.append( en_col + '_nnq_of_' + use_col )
    return data

# for en_col in use_cate:
#     for use_col in cat_feats:
#         if en_col != use_col:
#             data = nnq_encode(data,en_col,use_col)
# for en_col in use_cate:
#     for use_col in use_cate:
#         if en_col != use_col:
#             colname =  en_col +'_count_' + use_col
#             features.append( colname)
#             data[ colname] = data[en_col].astype(str) + "|" + data[use_col].astype(str)
#             data[ colname] = data.groupby([ colname ])[colname].transform('count')
# for en_col in use_cate:
#     for use_col in all_feat:
#         if en_col != use_col:
#             colname =  en_col +'_mean_' + use_col
#             features.append( colname)
#             data[ colname] = data[en_col].map( data.groupby([en_col])[use_col].mean() )
#             colname =  en_col +'_std_' + use_col
#             features.append( colname)
#             data[ colname] = data[en_col].map( data.groupby([en_col])[use_col].std() )


In [6]:
classMap = {'Excellent':0,
'Good':1,
'Pass':2,
'Fail':3,}
tr_index = ~data.Quality_label.isnull()
train_df = data[tr_index][use_cate +['Quality_label']].reset_index(drop=True)
train_df['Quality_label'] = train_df['Quality_label'].map(classMap)

test_df = data[~tr_index].reset_index(drop=True)
id_test = test_df.Group.values


X_train = train_df[use_cate]    
y = train_df.Quality_label
X_test = test_df[use_cate]
del train,test
del train_df,test_df
gc.collect()

87

In [7]:
X_train[:10]

Parameter1  Parameter10  Parameter2  Parameter3  Parameter4  Parameter5  \
0      0.001660     1.010385    0.591013  147.608373   38.186345    0.000421   
1      1.601749     1.010385    0.015052    0.035864   51.130326    0.000909   
2      0.098039     1.010385   69.233685    0.080920    0.112265    0.000909   
3     18.181860     1.010385    0.047325    0.018061    1.098102    0.000909   
4      0.012085     1.010385    0.008749    0.005509  524.327396    0.000909   
5      0.004062     1.010385   14.556483    0.786945    0.010545    0.000525   
6      0.438449     0.010192    1.232559    2.882699    0.610757    1.600654   
7  48159.917401     0.010192    0.002987   14.863813    0.063287    1.434060   
8      1.456601     0.010192   52.381578   20.625283  366.074831    1.434060   
9      0.000109     0.010192    4.784654   31.916672   20.672388    1.284806   

   Parameter6   Parameter7  Parameter8  Parameter9  
0    0.000612  2286.523413    0.035407    0.593081  
1    0.002397  2286.523413    0.035407    0.593081  
2    0.001972  2286.523413    0.035407    0.593081  
3    0.002397  2286.523413    0.035407    0.593081  
4    0.002397  2286.523413    0.035407    0.593081  
5    0.001623  2286.523413    0.035407    0.593081  
6    0.464037     0.600827   17.850021    0.051850  
7    0.314162     0.600827   17.850021    0.051850  
8    0.258497     0.600827   17.850021    0.051850  
9    0.175007     0.600827   17.850021    0.051850

In [9]:
lgb_paras = {
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'learning_rate': 0.01,
        'num_leaves': 32,
        # 'lambda_l1': 0.01,
        # 'lambda_l2': 10,
        'num_class': 4,
        'max_depth': -1,
        'seed': 2019,
        'feature_fraction': 0.8,
        #         'bagging_fraction': 0.8,
        #         'bagging_freq': 4,
        'verbose': 1,
        'device': 'gpu',
        'gpu_device_id':'0',
    }
from sklearn.metrics import f1_score, confusion_matrix
all_preads = []
skf = StratifiedKFold(n_splits=5, random_state=2019, shuffle=True)
def f1_weighted(preds, train_data):
    y_true = train_data.label
    preds = np.argmax(preds.reshape(4, -1), axis=0)
    score = f1_score(y_true, preds, average='weighted')
    return 'f1_weighted', score, True
evals_result = {}
for index, (train_index, test_index) in enumerate(skf.split(X_train, y)):
    train_x, test_x, train_y, test_y = X_train[use_cate].iloc[train_index], X_train[use_cate].iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    dtrain = lgb.Dataset(train_x, train_y)

    dvalid = lgb.Dataset(test_x, test_y)
#     clf = lgb.train(
#         params=lgb_paras,
#         train_set=dtrain,
#         num_boost_round=10000,
#         valid_sets=[dtrain, dvalid],
#         evals_result=evals_result,
#         early_stopping_rounds=100,
#         valid_names=["train", "valid"],
#         # categorical_feature=cate_cols,
#         verbose_eval=100)
    
    lgb_modelall = lgb.train(lgb_paras, dtrain,
                             valid_sets=[dtrain, dvalid],
                             num_boost_round=1000,
                             early_stopping_rounds=100,
                             valid_names=["train", "valid"],
                             evals_result=evals_result,
                             verbose_eval=50,
                             feval=f1_weighted)
    pred = lgb_modelall.predict(X_test)
    all_preads.append( pred )


    

Training until validation scores don't improve for 100 rounds.
[50]	train's multi_logloss: 1.17429	train's f1_weighted: 0.298003	valid's multi_logloss: 1.21025	valid's f1_weighted: 0.263807
[100]	train's multi_logloss: 1.09496	train's f1_weighted: 0.545342	valid's multi_logloss: 1.16197	valid's f1_weighted: 0.443071
[150]	train's multi_logloss: 1.0392	train's f1_weighted: 0.59546	valid's multi_logloss: 1.13434	valid's f1_weighted: 0.476635
[200]	train's multi_logloss: 0.998146	train's f1_weighted: 0.618237	valid's multi_logloss: 1.11962	valid's f1_weighted: 0.496565
[250]	train's multi_logloss: 0.963889	train's f1_weighted: 0.631733	valid's multi_logloss: 1.11221	valid's f1_weighted: 0.494446
[300]	train's multi_logloss: 0.934651	train's f1_weighted: 0.645498	valid's multi_logloss: 1.10827	valid's f1_weighted: 0.49552
[350]	train's multi_logloss: 0.908083	train's f1_weighted: 0.659653	valid's multi_logloss: 1.10586	valid's f1_weighted: 0.496744
Early stopping, best iteration is:
[270]	

In [11]:
mean_pread = np.mean(all_preads,axis=0)
cols = ['Excellent ratio','Good ratio','Pass ratio','Fail ratio']

sub_prob  = pd.DataFrame(mean_pread,columns=cols)
len(sub_prob),len(id_test)
sub_prob['Group'] = id_test

sub_prob['Group'] = sub_prob['Group'].map(int)
sub_prob = sub_prob.groupby([ 'Group'])[cols].median().reset_index()

sub_prob.to_csv('../submission/lgb_test.csv',index=False)

In [12]:
all_preads[0][0]

array([0.47844266, 0.26459496, 0.1909932 , 0.06596918])

In [13]:
features

['Parameter1',
 'Parameter10',
 'Parameter2',
 'Parameter3',
 'Parameter4',
 'Parameter5',
 'Parameter6',
 'Parameter7',
 'Parameter8',
 'Parameter9']